# CORD-19-collect-scopus-data

In general, this jupyter notebook is designated to collect additional data via scopus to enbroaden the CORD19 dataset: 
https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook.

In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

Get the data and save it to a variable.

In [2]:
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv')

Check the length of the column containing doi's.

In [3]:
len(CORD19_CSV['doi'])

77448

Display the column doi to see if there are inconsistencies such as NaN's

In [4]:
doi = CORD19_CSV['doi']
doi

0                                 NaN
1          10.1016/j.regg.2021.01.002
2           10.1016/j.rec.2020.08.002
3        10.1016/j.vetmic.2006.11.026
4                   10.3390/v12080849
                     ...             
77443      10.1007/s11229-020-02869-9
77444                             NaN
77445     10.1101/2020.05.13.20100206
77446      10.1007/s42991-020-00052-8
77447     10.1101/2020.09.14.20194670
Name: doi, Length: 77448, dtype: object

Create a series with solely unique values and neglect NaN's. It is important to sort the unique values. Otherwise, the method is creating different results after each restart of the notebook. 

In [5]:
doi_counted = doi.value_counts().sort_index(ascending=True)
doi_counted

10.1001/jamainternmed.2020.1369       1
10.1001/jamanetworkopen.2020.16382    1
10.1001/jamanetworkopen.2020.17521    1
10.1001/jamanetworkopen.2020.20485    1
10.1001/jamanetworkopen.2020.24984    1
                                     ..
10.9745/ghsp-d-20-00115               1
10.9745/ghsp-d-20-00171               1
10.9745/ghsp-d-20-00218               1
10.9758/cpn.2020.18.4.607             1
10.9781/ijimai.2020.02.002            1
Name: doi, Length: 74302, dtype: int64

The following function determines the requested information from the Scopus API. (https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4)

In [6]:
#Adapted from https://github.com/ElsevierDev/elsapy/blob/master/exampleProg.py
def fetch_scopus_api(client, doi):
    """obtain additional paper information from scopus by doi
    """
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    #print ("doc_srch has", len(doc_srch.results), "results.")
    #print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

Thusly, the configuration file is set up and contains an APIkey. Further information: https://github.com/ElsevierDev/elsapy/blob/master/CONFIG.md

In [7]:
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

Moreover, the client is initialized with the API-Key.

In [8]:
client = ElsClient(config['apikey'])

For demonstation purposes, the following cells shows which data is returned by the Scopus API. 

In [9]:
return_example = fetch_scopus_api(client, '10.1016/j.dsx.2020.04.012')
print(json.dumps(return_example, indent=2))

{
  "affiliation": [
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Hamdard",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Millia Islamia",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Indraprastha Apollo Hospitals",
      "affiliation-country": "India"
    }
  ],
  "coredata": {
    "srctype": "j",
    "eid": "2-s2.0-85083171050",
    "pubmed-id": "32305024",
    "prism:coverDate": "2020-07-01",
    "prism:aggregationType": "Journal",
    "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85083171050",
    "dc:creator": {
      "author": [
        {
          "ce:given-name": "Raju",
          "preferred-name": {
            "ce:given-name": "Raju",
            "ce:initials": "R.",
            "ce:surname": "Vaishya",
            "ce:indexed-name": "Vaishya R."
          },
          "@seq": "1",
          "ce:init

Based on the returned data, further analysis is conductable. Therefore, two notebooks are created to analyse data linked to: 
<ul>
  <li>affiliation</li>
  <li>coredata</li>
</ul>    

Thusly, the already fetched coredata and affiliation are read and combined to a DataFrame for further processing.

In [10]:
df_current_extra_info = pd.DataFrame()
try:
    read_affiliation = pd.read_pickle('extra_info_affiliation_CS.pkl')
    read_coredata = pd.read_pickle('extra_info_coredata_CS.pkl')
    df_current_extra_info['affiliation'] = read_affiliation
    df_current_extra_info['coredata'] = read_coredata
    df_current_extra_info
except:
    print("The DataFrame is empty")
    #if the dataframe is not empty set the variable to show the dataframe

The length of the DataFrame containing the current information is assigned to a variable to be used for further processing. 
Therefore, the length will be used within a while loop as a starting index. 

In [11]:
len_df_current_extra_info = len(df_current_extra_info)
len_df_current_extra_info

74302

In [12]:
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': 'Palo Alto', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083266658', ..."
1,"[{'affiliation-city': 'Seattle', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
2,"[{'affiliation-city': 'Cambridge', 'affilname'...","{'srctype': 'j', 'prism:issueIdentifier': '8',..."
3,"[{'affiliation-city': 'Madison', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '9',..."
4,"[{'affiliation-city': 'Los Angeles', 'affilnam...","{'srctype': 'j', 'prism:issueIdentifier': '11'..."
...,...,...
74297,"{'affiliation-city': 'Rockville', 'affilname':...","{'srctype': 'j', 'eid': '2-s2.0-85092678139', ..."
74298,"[{'affiliation-city': 'Glastonbury', 'affilnam...","{'srctype': 'j', 'eid': '2-s2.0-85087468210', ..."
74299,"{'affiliation-city': 'Newark', 'affilname': 'U...","{'srctype': 'j', 'eid': '2-s2.0-85092677974', ..."
74300,"[{'affiliation-city': 'Istanbul', 'affilname':...","{'srctype': 'j', 'prism:issueIdentifier': '4',..."


In [13]:
def contains_only_None(dic):
    """
    This functions inspects an dictionary and returns True if it solely contains None values
    """
    return len(dic) == sum(value == None for value in dic.values())    

In [14]:
def append_fetched_data_to_df(df_current_extra_info, dic):
    """
    This function appends or inserts newly fetched data to the DataFrame containing scopus data.
    """
    #df_current_extra_info -> holding the latest data, new data needs to be appended to it, 
    #df_newly_fetched_transposed -> holdy newly fetched data, needs to be inserted or fetched
    
    if contains_only_None(dic):
        placeholder_entries = pd.DataFrame(np.empty((len(dict_new_extra_info),2),dtype=object),columns=['affiliation','coredata'], index=dict_new_extra_info.keys())
        df_newly_fetched_transposed = placeholder_entries
        print(placeholder_entries)
    else:
        #Prior appending, the dictionary is transformed to a DataFrame
        df_newly_fetched = pd.DataFrame(dic)
        #For readability, the DataFrame is transposed
        df_newly_fetched_transposed = df_newly_fetched.T
        print(df_newly_fetched_transposed)
    
    #Insert newly fetched rows which were previously not successful appended
    for index, row in df_newly_fetched_transposed.iterrows():
        #insert to current extra info DataFrame because the row is existent
        if index in df_current_extra_info.index and row.affiliation is not None:
            df_current_extra_info.loc[index] = row
        #append to current extra info DataFrame because the row is new     
        if index not in df_current_extra_info.index:
            df_current_extra_info = df_current_extra_info.append(row, ignore_index=True)
            
    #returning DataFrame with inserted and replaced rows. 
    return df_current_extra_info

Both Dataframes columns are stored each to an object. The series objects are stored to each to a pkl-file which is not exceeding the size of 100MB allowing GitHub uploads.

In [15]:
def store_df_columns(df):
    ser_affiliation = df['affiliation']
    ser_coredata = df['coredata']
    ser_affiliation.to_pickle('extra_info_affiliation_CS.pkl')
    ser_coredata.to_pickle('extra_info_coredata_CS.pkl')
    return ser_affiliation, ser_coredata

In [16]:
# placeholder_entries = pd.DataFrame(np.empty((4,2),dtype=object),columns=['affiliation','coredata'])

In [17]:
# placeholder_entries

Subsequently, the fetched scopus data is stored within a dictionary. Besides, the print function is used to show the state of the process by displaying the latest fetched information. 

In [18]:
%%time
dict_new_extra_info = dict()
len_dois = len(doi_counted)
def trigger_fetching():
    threshold = 0 
    i = len_df_current_extra_info
    while i < len_dois: #-> upto modified, normally len_dois
        dict_new_extra_info[i] = fetch_scopus_api(client, doi_counted.index[i])
        print("Position fetched: " + str(i) + " -> " +  doi_counted.index[i])
        i = i + 1 
#         threshold = threshold + 1
#         if threshold > 99:
#             df_combined_extra_info = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info)
#             stored_series = store_df_columns(df_combined_extra_info)
#             threshold = 0
#             print("batch saved")
trigger_fetching()

Wall time: 0 ns


The following cell is useful when the process above is interrupted. Therefore, the dictionary containing fetched information can be narrowed down to useful entries. 

In [19]:
# def save_new_extra_info(len_df_current_extra_info, upto):
#     """
#     This function is used to separate successfull API calls from API calls which were prevented due to an invalid API-Key.
#     As a result, this function returns a range of valid entries up to the given parameter. 
#     """
#     dict_new_extra_info_saver = dict()
#     i = len_df_current_extra_info
#     while i < upto:
#         #print("Position: " + str(i) + " -> " +  doi_counted.index[i])
#         dict_new_extra_info_saver[i] = dict_new_extra_info[i]
#         i = i + 1 
#     return dict_new_extra_info_saver

The existing and newly fetched information are combined into one DataFrame. 

In [20]:
df_combined_extra_info = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info)
df_combined_extra_info

Empty DataFrame
Columns: [affiliation, coredata]
Index: []


,affiliation,coredata
0,"[{'affiliation-city': 'Palo Alto', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083266658', ..."
1,"[{'affiliation-city': 'Seattle', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
2,"[{'affiliation-city': 'Cambridge', 'affilname'...","{'srctype': 'j', 'prism:issueIdentifier': '8',..."
3,"[{'affiliation-city': 'Madison', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '9',..."
4,"[{'affiliation-city': 'Los Angeles', 'affilnam...","{'srctype': 'j', 'prism:issueIdentifier': '11'..."
...,...,...
74297,"{'affiliation-city': 'Rockville', 'affilname':...","{'srctype': 'j', 'eid': '2-s2.0-85092678139', ..."
74298,"[{'affiliation-city': 'Glastonbury', 'affilnam...","{'srctype': 'j', 'eid': '2-s2.0-85087468210', ..."
74299,"{'affiliation-city': 'Newark', 'affilname': 'U...","{'srctype': 'j', 'eid': '2-s2.0-85092677974', ..."
74300,"[{'affiliation-city': 'Istanbul', 'affilname':...","{'srctype': 'j', 'prism:issueIdentifier': '4',..."


In [21]:
#to big for GitHub
#df_combined_extra_info.to_csv('extra_info_CS5099.csv', sep='\t')

Both Dataframes columns are stored each to an object. The series objects are stored to each to a pkl-file which is not exceeding the size of 100MB allowing GitHub uploads.

In [22]:
stored_series = store_df_columns(df_combined_extra_info)
stored_series[0]

0        [{'affiliation-city': 'Palo Alto', 'affilname'...
1        [{'affiliation-city': 'Seattle', 'affilname': ...
2        [{'affiliation-city': 'Cambridge', 'affilname'...
3        [{'affiliation-city': 'Madison', 'affilname': ...
4        [{'affiliation-city': 'Los Angeles', 'affilnam...
                               ...                        
74297    {'affiliation-city': 'Rockville', 'affilname':...
74298    [{'affiliation-city': 'Glastonbury', 'affilnam...
74299    {'affiliation-city': 'Newark', 'affilname': 'U...
74300    [{'affiliation-city': 'Istanbul', 'affilname':...
74301                                                 None
Name: affiliation, Length: 74302, dtype: object

In [23]:
stored_series[1]

0        {'srctype': 'j', 'eid': '2-s2.0-85083266658', ...
1        {'srctype': 'j', 'prism:issueIdentifier': '7',...
2        {'srctype': 'j', 'prism:issueIdentifier': '8',...
3        {'srctype': 'j', 'prism:issueIdentifier': '9',...
4        {'srctype': 'j', 'prism:issueIdentifier': '11'...
                               ...                        
74297    {'srctype': 'j', 'eid': '2-s2.0-85092678139', ...
74298    {'srctype': 'j', 'eid': '2-s2.0-85087468210', ...
74299    {'srctype': 'j', 'eid': '2-s2.0-85092677974', ...
74300    {'srctype': 'j', 'prism:issueIdentifier': '4',...
74301                                                 None
Name: coredata, Length: 74302, dtype: object

Verifying that the returned None values are due to non existent data and not to an invalid API-Key

In [24]:
def enrich_data():
    """
    This function fetches again the scopus API and solely asks for information which previously returned None. 
    """
    #Add a new column to the DataFrame containg the DOI's which are used to fetch the API
    ser_doi = pd.Series(doi_counted.index[:len_data])
    df_current_extra_info_checker = df_combined_extra_info
    df_current_extra_info_checker['doi'] = ser_doi
    
    #fetching solely none entries 
    len_df_current_extra_info_checker = len(df_current_extra_info_checker)
    dict_new_extra_info_checker = dict()
    len_df_current_extra_info_checker = 70000
    i = 60000 
    while i < len_df_current_extra_info_checker: 
        if df_current_extra_info_checker['affiliation'][i] == None:
            dict_new_extra_info_checker[i] = fetch_scopus_api(client, ser_doi[i])
            print("Position fetched again: " + str(i) + " -> " +  ser_doi[i])
        i = i + 1
    #check if at least one value is not None, otherwise the process is finished
    if contains_only_None(dict_new_extra_info_checker):
        print("The scopus API did not returned new information for existing None values.")
    else:
        #There is new information to insert to the existing DataFrame
        df_combined_extra_info_fetched_again  = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info_checker)
        store_df_columns(df_combined_extra_info_fetched_again)
        # Show an extract of the newly fetched and inserted data
#         df_combined_extra_info_fetched_again['check_doi'] = ser_doi
#         df_combined_extra_info_fetched_again.head(30)

In [25]:
len_dois = len(doi_counted)
len_dois

74302

In [26]:
len_data = len(stored_series[0])
len_data 

74302

In [ ]:
if len_dois == len_data:
    enrich_data()
else:
    print("There are entries which are not yet fetched from the scopus API.")

Position fetched again: 60040 -> 10.12965/jer.150267
Position fetched again: 60042 -> 10.12998/wjcc.v5.i6.212
Position fetched again: 60056 -> 10.12998/wjcc.v8.i21.5188
Position fetched again: 60057 -> 10.12998/wjcc.v8.i21.5250
Position fetched again: 60070 -> 10.1302/2046-3758.16.bjo-2020-0040.r1
Position fetched again: 60071 -> 10.1302/2046-3758.16.bjo-2020-0041.r1
Position fetched again: 60072 -> 10.1302/2046-3758.16.bjo-2020-0059.r1
Position fetched again: 60073 -> 10.1302/2046-3758.16.bjo-2020-0063.r1
Position fetched again: 60076 -> 10.1302/2633-1462.110.bjo-2020-0120.r1
Position fetched again: 60077 -> 10.1302/2633-1462.110.bjo-2020-0126.r1
Position fetched again: 60078 -> 10.1302/2633-1462.110.bjo-2020-0149.r1
Position fetched again: 60079 -> 10.1302/2633-1462.111.bjo-2020-0095.r1
Position fetched again: 60080 -> 10.1302/2633-1462.112.bjo-2020-0144.r1
Position fetched again: 60081 -> 10.1302/2633-1462.15.bjo-2020-0019
Position fetched again: 60082 -> 10.1302/2633-1462.15.bjo-20

Position fetched again: 63969 -> 10.18772/26180197.2020.v2nsia6
Position fetched again: 63974 -> 10.1890/1051-0761(2000)010[0115:snzfdw]2.0.co;2
Position fetched again: 63977 -> 10.19104/jepm.2016.116
Position fetched again: 63986 -> 10.20524/aog.2020.0527
Position fetched again: 63987 -> 10.20524/aog.2020.0546
Position fetched again: 63988 -> 10.20524/aog.2020.0547
Position fetched again: 63989 -> 10.20900/jpbs.20200024
Position fetched again: 63990 -> 10.20944/preprints202003.0309.v1
Position fetched again: 63991 -> 10.20944/preprints202005.0318.v1
Position fetched again: 63992 -> 10.20944/preprints202010.0279.v1
Position fetched again: 63996 -> 10.21053/ceo.2020.00570
Position fetched again: 63997 -> 10.21053/ceo.2020.00612
Position fetched again: 64009 -> 10.2106/jbjs.oa.19.00050
Position fetched again: 64010 -> 10.21106/ijma.378
Position fetched again: 64011 -> 10.21106/ijma.389
Position fetched again: 64012 -> 10.21106/ijma.394
Position fetched again: 64013 -> 10.21106/ijma.401
P

Position fetched again: 64595 -> 10.2196/19371
Position fetched again: 64607 -> 10.2196/20596
Position fetched again: 64610 -> 10.2196/20735
Position fetched again: 64635 -> 10.2196/24388
Position fetched again: 64647 -> 10.2196/25995
Position fetched again: 64660 -> 10.22133/ijwr.2020.250275.1070
Position fetched again: 64662 -> 10.2217/3dp-2020-0013
Position fetched again: 64663 -> 10.2217/3dp-2020-0014
Position fetched again: 64666 -> 10.2217/crc-2020-0018
Position fetched again: 64723 -> 10.22540/jfsf-05-086
Position fetched again: 64725 -> 10.2307/1936838
Position fetched again: 64727 -> 10.2307/1942165
Position fetched again: 64728 -> 10.2307/1942442
Position fetched again: 64793 -> 10.23937/2469-5858/1510092
Position fetched again: 64826 -> 10.2478/jtim-2020-0010
Position fetched again: 64827 -> 10.2478/jtim-2020-0018
Position fetched again: 64848 -> 10.2490/prm.20200018
Position fetched again: 64853 -> 10.25100/cm.v51i3.4534
Position fetched again: 64857 -> 10.25147/ijcsr.2017.

Position fetched again: 67773 -> 10.3389/fpubh.2015.00264
Position fetched again: 67776 -> 10.3389/fpubh.2017.00121
Position fetched again: 67779 -> 10.3389/fpubh.2018.00174
Position fetched again: 67826 -> 10.3389/fpubh.2020.00360
Position fetched again: 67842 -> 10.3389/fpubh.2020.00426
Position fetched again: 68018 -> 10.3389/frcmn.2020.583376
Position fetched again: 68031 -> 10.3389/fspor.2020.00098
Position fetched again: 68032 -> 10.3389/fspor.2020.00112
Position fetched again: 68033 -> 10.3389/fspor.2020.579421
Position fetched again: 68034 -> 10.3389/fspor.2020.579830
Position fetched again: 68035 -> 10.3389/fspor.2020.586141
Position fetched again: 68036 -> 10.3389/fspor.2020.589214
Position fetched again: 68037 -> 10.3389/fspor.2020.589483
Position fetched again: 68038 -> 10.3389/fspor.2020.589702
Position fetched again: 68039 -> 10.3389/fspor.2020.589813
Position fetched again: 68040 -> 10.3389/fspor.2020.590151
Position fetched again: 68041 -> 10.3389/fspor.2020.593499
Posi

In [ ]:
# ser_doi = pd.Series(doi_counted.index[:len_data])
# ser_doi

In [ ]:
# df_current_extra_info_checker = df_combined_extra_info
# df_current_extra_info_checker['doi'] = ser_doi

In [ ]:
# %%time
# len_df_current_extra_info_checker = len(df_current_extra_info_checker)
# dict_new_extra_info_checker = dict()
# i = 0 
# while i < len_df_current_extra_info_checker: 
#     if df_current_extra_info_checker['affiliation'][i] == None:
#         dict_new_extra_info_checker[i] = fetch_scopus_api(client, ser_doi[i])
#         print("Position fetched again: " + str(i) + " -> " +  ser_doi[i])
#     i = i + 1    

In [ ]:
# dict_new_extra_info_checker
# -> check if at least one value is not None -> otherwise the process is finished here

In [ ]:
# len(dict_new_extra_info_checker)

In [ ]:
# df_combined_extra_info_fetched_again  = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info_checker)
# df_combined_extra_info_fetched_again

In [ ]:
# store_df_columns(df_combined_extra_info_fetched_again)

In [ ]:
# df_combined_extra_info_fetched_again['check_doi'] = ser_doi
# df_combined_extra_info_fetched_again.head(30)